url = "https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py"

In [1]:
import datetime
import matplotlib.pyplot as plt
import os

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

# 设置超参数

In [1]:
batch_size, num_workers = 8, 2
num_epochs = 100
lr = 0.1

# 模型

## ResNet

In [3]:
class BasicBlock(nn.Module):
    #在resnet18、34中使用的残差块
    expansion=1
    def __init__(self, inplanes, planes, stride=1):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(
            inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False,
        )
        self.bn1 = nn.BatchNorm2d(planes)
        
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False,
        )
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        self.downsample = nn.Sequential()
        if stride != 1 or inplanes != self.expansion*planes:
            self.downsample = nn.Sequential(
                nn.Conv2d(
                    inplanes, self.expansion*planes, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(self.expansion*planes)
            )
    def forward(self, x):
        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.downsample(x)
        out = self.act(out)
        return out

def test_basicblock():
    inplanes, planes = 16, 16
    x = torch.randn(1, inplanes, 64, 64)
    y = BasicBlock(inplanes, planes, stride=1)(x)
    return y.shape

test_basicblock()

torch.Size([1, 16, 64, 64])

In [4]:
class Bottlennck(nn.Module):
    #在resnet50、101、152中使用的残差块
    expansion = 4
    def __init__(self, inplanes, planes, stride=1):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(
            inplanes, planes, kernel_size=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3,stride=stride, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.conv3 = nn.Conv2d(
            planes, self.expansion*planes, kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.downsample = nn.Sequential()
        if stride != 1 or inplanes != self.expansion*planes:
            self.downsample = nn.Sequential(
                nn.Conv2d(
                    inplanes, self.expansion*planes, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(self.expansion*planes),
            )
    def forward(self, x):
        out = self.act(self.bn1(self.conv1(x)))
        out = self.act(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.downsample(x)
        out = self.act(out)
        return out
    
def test_bottlennck():
    inplanes, planes = 16, 16
    x = torch.randn(1, inplanes, 64, 64)
    y = Bottlennck(inplanes, planes, stride=1)(x)
    return y.shape

test_bottlennck()

torch.Size([1, 64, 64, 64])

In [5]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.inplanes = 64
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.layer1 = self._make_layer(
            block, 64, num_blocks[0], stride=1
        )
        self.layer2 = self._make_layer(
            block, 128, num_blocks[1], stride=2
        )
        self.layer3 = self._make_layer(
            block, 256, num_blocks[2], stride=2
        )
        self.layer4 = self._make_layer(
            block, 512, num_blocks[3], stride=2
        )
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(512*block.expansion, num_classes)
    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.inplanes, planes, stride))
            self.inplanes = planes*block.expansion
        return nn.Sequential(*layers)
    def forward(self, x):
        out = self.stem(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18(num_classes=10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

def ResNet34(num_classes=10):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes)

def ResNet50(num_classes=10):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes)

def ResNet101(num_classes=10):
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=num_classes)

def ResNet152(num_classes=10):
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes=num_classes)

def test():
    x = torch.randn(1, 3, 32, 32)
    y = ResNet18()(x)
    return y.shape
test()

torch.Size([1, 10])

# 数据集

In [7]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4882, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4882, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )
])

In [8]:
train_set = torchvision.datasets.CIFAR10(
    root="./data",train=True, transform=train_transform
)
test_set = torchvision.datasets.CIFAR10(
    root="./data",train=False, transform=test_transform
)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

# 函数

In [9]:
def train(net, train_loader, test_loader, num_epochs, lr): 
    net = nn.DataParallel(net)
    cudnn.benchmark = True
    best_acc = 0 # 用以保存最好的模型结果
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(
        net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs
    )
    for epoch in range(num_epochs):
        net.train()
        train_loss, train_corrent, test_corrent = 0, 0, 0
        start_time = datetime.datetime.now()
        for i, data in enumerate(train_loader):
            img, label = data
            img, label = img.cuda(), label.cuda()
            optimizer.zero_grad()
            outputs = net(img)
            loss = loss_func(outputs, label)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, pred = torch.max(outputs, 1)
            train_corrent += (pred==label).sum()
        end_time = datetime.datetime.now()
        train_batch_time = (end_time-start_time).seconds / len(train_loader.dataset) * batch_size # 每个batch_size训练时间
        train_acc  = train_corrent / len(train_loader.dataset) * 100.
        #############测试################
        net.eval()
        start_time = datetime.datetime.now()
        for i, data in enumerate(test_loader):
            img, label = data
            img, label = img.cuda(), label.cuda()
            outputs = net(img)
            _, pred = torch.max(outputs, 1)
            test_corrent += (pred==label).sum()
        end_time = datetime.datetime.now()
        test_batch_time = (end_time-start_time).seconds / len(test_loader.dataset) * batch_size
        test_acc = test_corrent / len(test_loader.dataset) * 100.
        scheduler.step()
        print("epoch:{}|{}: loss: {:.3f}, train_acc: {:.3f}, test_acc: {:.3f}, time: {:.3f}|{:.3f}".format(epoch+1, num_epochs, train_loss, train_acc, test_acc, train_batch_time, test_batch_time))
        if test_acc > best_acc:
            state = {
                "net": net.state_dict(),
                "best_acc": test_acc,
                "epoch": epoch
            }
            if not os.path.isdir("cifar10_resnet_checkpoint"):
                os.mkdir("cifar10_resnet_checkpoint")
            torch.save(state, "./cifar10_resnet_checkpoint/best_ckpt.pth")
            best_acc = test_acc

# 训练

In [ ]:
train(net=ResNet18(), train_loader=train_loader, test_loader=test_loader, num_epochs=num_epochs, lr=lr)

0|5: loss: 12455.9325928092, train_acc: 26.76799964904785, test_acc: 34.790000915527344, time: 0.08208|0.0168
1|5: loss: 9985.866578131914, train_acc: 41.95399856567383, test_acc: 52.59000015258789, time: 0.08176|0.0176
2|5: loss: 7930.917583227158, train_acc: 55.40800094604492, test_acc: 64.16999816894531, time: 0.08192|0.0168
3|5: loss: 6371.415918037295, train_acc: 64.63600158691406, test_acc: 69.54000091552734, time: 0.08256|0.0176
